# Spotify Afrobeats Recommendation System
By Afolabi Cardoso

## Data Gathering

This notebook features the data gathering process. Using the Spotify API and Spotipy library, I was able to create functions to collect the useful metadata and audio features from the tracks in the playlist. After collecting the features as a dataframe, I exported them as a csv

#### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

import time

#### Authentication

To gain access to the Spotify API, I need to create a client id and client secret.

In [ ]:

client_credentials_manager = SpotifyClientCredentials(client_id = '2101cd224f5948e19c4c782d76744ed3',
                                                      client_secret = '879abdfca432449facc9d8566fb40ab6')

#### Spotipy Object

Using the client id and client secret created above, I create a spotipy Object using SpotifyClientCredentials

In [ ]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

#### Get track metadata and features

This function takes in a playlist URI and returns a dataframe containing metadata such as artist name, genre, album name, track_uri, danceability, energy, loudness, instrumentalness etc

In [ ]:
def get_track_info(playlist):
    
    #split the playlist_uri is at the end of the playlists url. I'll use the .split method to extract it
    uri = playlist.split("/")[-1].split("?")[0]
    
    #from the spotipy library, use the playlist_tracks() method to extract each track from the playlist uri
    #It comes in a nested dictionary format
    
    results = sp.playlist_tracks(uri)
    tracks = results['items']
    
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    
    
    #create an empty dictionary with the info we want to extract as columns
    info = {
    'track_uri':[],
    'track_name':[],
    'artist_name':[],
    'artist_info':[],
    'artist_uri':[],
    'artist_popularity':[],
    'artist_genre':[],
    'album':[],
    'track_pop':[],
    'year_released':[]
    }
    
    features = {'danceability': [],
     'energy': [],
     'key': [],
     'loudness': [],
     'mode': [],
     'speechiness': [],
     'acousticness': [],
     'instrumentalness': [],
     'liveness': [],
     'valence': [],
     'tempo': [],
     'type': [],
     'id': [],
     'uri': [],
     'track_href': [],
     'analysis_url': [],
     'duration_ms': [],
     'time_signature': []
               }
    
    #using a for loop, get the the info for each song and put it into the empty dictionary
    for track in tracks:
        #URI
        info['track_uri'].append((track["track"]["uri"]).split(':')[2])

        #Track name
        info['track_name'].append(track["track"]["name"])

        #Main Artist
        info['artist_uri'].append((track["track"]["artists"][0]["uri"]).split(':')[2])
        info['artist_info'].append(sp.artist(track["track"]["artists"][0]["uri"]))

        #Name, popularity, genre
        info['artist_name'].append(track["track"]["artists"][0]["name"])
        info['artist_popularity'].append(sp.artist(track["track"]["artists"][0]["uri"])["popularity"])
        info['artist_genre'].append(sp.artist(track["track"]["artists"][0]["uri"])["genres"])

        #Album
        info['album'].append(track["track"]["album"]["name"])

        #Popularity of the track, year released
        info['track_pop'].append(track["track"]["popularity"])
        info['year_released'].append(track["track"]['album']['release_date'])
        
        #Transform the info dictionary into a dataframe
        info_df = pd.DataFrame(info)
        
        #loop through the tracks to their features and assign it to the empty dictionary
        track_uri = track["track"]["uri"].split(':')[2] 
        
        try:
            for key,value in (sp.audio_features(track_uri)[0]).items():
                features[key].append(value)
            
        except:
            print(f'failed on track {track["track"]["name"]}')
            continue
        time.sleep(3)
        
    #Transform the features dictionary into a dataframe
    features_df = pd.DataFrame(features)
    
        
    
    return info_df.join(features_df)
        

#### Add label to the playlist dataframe


This function calls the get_track_info function above and creates a column with the playlist genre or given username

In [ ]:
def raw_data(user_playlist_url, genre):
    user_playlist_info = get_track_info(user_playlist_url)
    #add user genre
    user_playlist_info.loc[:,'genre'] = genre
    return user_playlist_info

#### Afrobeats playlist

I created an afrobeats playlist with spotify that contains 1500+ tracks. Using the functions above, I will fetch the tracks metadata and features

In [ ]:
afrobeats_playlist_url= "https://open.spotify.com/playlist/5ZCzd0nCLqiIX1jwQWfazW"

In [ ]:
afrobeats_df = raw_data(afrobeats_playlist_url, 'afrobeats')

In [ ]:
afrobeats_df.head(2)

In [ ]:
len(afrobeats_df)

In [ ]:
afrobeats_df.isna().sum().sum()

In [ ]:
afrobeats_df[afrobeats_df['energy'].isnull()]

In [ ]:
afrobeats_df.dropna(inplace=True)

In [ ]:
afrobeats_df.isnull().sum().sum()

#### Exporta afrobeats as a csv

In [ ]:
afrobeats_df.to_csv('../data/afrobeats.csv', index = False)

#### Jacks playlist

My classmate Jack volunteered his spotify playlist which contains mostly classical music. 

In [ ]:
jacks_playlist_url = 'https://open.spotify.com/playlist/6UlskZAcTPzcGMnQaMnIVm?si=5cbb031d1fdd4064'

In [ ]:
jacks_playlist_df = raw_data(jacks_playlist_url, 'jack')

In [ ]:
jacks_playlist_df.head()

#### Exporta Jack's playlist as a csv

In [ ]:
jacks_playlist_df.to_csv('../data/jack.csv', index = False)

#### Ankita playlist

Ankita, also a classmate, gave me her spotify url. Her playlist is a mixture of pop and Indian music

In [ ]:
ankita_playlist_url = 'https://open.spotify.com/playlist/6qzbkhrmxdFj5TtaXR0sfI?si=ElY40mMjQ7apOsHTGlQI7A'

In [ ]:
ankita_playlist_df = raw_data(ankita_playlist_url, 'ankita')

In [ ]:
ankita_playlist_df.head()

#### Export Ankita's playlist as a csv

In [ ]:
ankita_playlist_df.to_csv('../data/ankita.csv', index = False)

#### Playlist featuring top songs from Fela

Fela pioneered the Afrobeats sound. He called his sound Afrobeat (without the s). I will compare the difference in sounds in the EDA section

In [ ]:
fela_playlist_url = 'https://open.spotify.com/playlist/1bSsiqaBobgbnZTTVEo4Qh?si=3093a84902434561'

In [ ]:
fela_playlist_df = raw_data(fela_playlist_url, 'fela')

In [ ]:
fela_playlist_df.head(2)

#### Exporta fela playlist as a csv

In [ ]:
fela_playlist_df.to_csv('../data/fela.csv', index = False)

#### Heavy metal playlist

In [ ]:
heavy_metal_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DX9qNs32fujYe?si=8686a89d3fb7415b'

In [ ]:
heavy_metal_playlist_df = raw_data(heavy_metal_playlist_url, 'heavy_metal')

In [ ]:
heavy_metal_playlist_df.head(2)

#### Export heavy metal playlist as a csv

In [ ]:
heavy_metal_playlist_df.to_csv('../data/heavymetal.csv', index = False)

#### Rock playlist

In [ ]:
rock_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DXcF6B6QPhFDv?si=53be43146b8f4585'

In [ ]:
rock_playlist_df = raw_data(rock_playlist_url, 'rock')

In [ ]:
rock_playlist_df.head(2)

#### Export rock playlist as a csv

In [ ]:
rock_playlist_df.to_csv('../data/rock.csv', index = False)

#### Jazz playlist

In [ ]:
jazz_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DXdwTUxmGKrdN?si=a054d3423fca4687'

In [ ]:
jazz_playlist_df = raw_data(jazz_playlist_url, 'jazz')

In [ ]:
jazz_playlist_df.head(2)

#### Export jazz playlist as a csv

In [ ]:
jazz_playlist_df.to_csv('../data/jazz.csv', index = False)

#### Country playlist

In [ ]:
country_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DWXdiK4WAVRUW?si=56683bfa3aca4788'

In [ ]:
country_playlist_df = raw_data(country_playlist_url, 'country')

In [ ]:
country_playlist_df.head(2)

#### Export country playlist as a csv

In [ ]:
country_playlist_df.to_csv('../data/country.csv', index = False)

#### Hip Hop playlist

In [ ]:
hiphop_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DX97h7ftpNSYT?si=5b73c05e83c9410f'

In [ ]:
hiphop_playlist_df = raw_data(hiphop_playlist_url, 'hiphop')

In [ ]:
hiphop_playlist_df.head(2)

#### Export Hip Hop playlist as a csv

In [ ]:
hiphop_playlist_df.to_csv('../data/hiphop.csv', index = False)

#### RnB playlist

In [ ]:
rnb_playlist_url = 'https://open.spotify.com/playlist/37i9dQZF1DWXbttAJcbphz?si=aa6c61a73e1a4c24'

In [ ]:
rnb_playlist_df = raw_data(rnb_playlist_url, 'rnb')

In [ ]:
rnb_playlist_df.head(2)

#### Export rnb playlist as a csv

In [ ]:
rnb_playlist_df.to_csv('../data/rnb.csv', index = False)

The next notebook features me performing exploratory data analysis on the csv exported. 